# Hybrid music recommender system

In [224]:
import numpy as np
import pandas as pd 

### Importing the data

In [225]:
users_history = pd.read_csv("/kaggle/input/million-song-dataset-spotify-lastfm/User Listening History.csv")
music_info  = pd.read_csv("/kaggle/input/million-song-dataset-spotify-lastfm/Music Info.csv")

In [226]:
users_history.shape

(9711301, 3)

In [227]:
music_info.shape

(50683, 21)

In [228]:
music_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50683 entries, 0 to 50682
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   track_id             50683 non-null  object 
 1   name                 50683 non-null  object 
 2   artist               50683 non-null  object 
 3   spotify_preview_url  50683 non-null  object 
 4   spotify_id           50683 non-null  object 
 5   tags                 49556 non-null  object 
 6   genre                22348 non-null  object 
 7   year                 50683 non-null  int64  
 8   duration_ms          50683 non-null  int64  
 9   danceability         50683 non-null  float64
 10  energy               50683 non-null  float64
 11  key                  50683 non-null  int64  
 12  loudness             50683 non-null  float64
 13  mode                 50683 non-null  int64  
 14  speechiness          50683 non-null  float64
 15  acousticness         50683 non-null 

In [229]:
users_history[1:4]

,track_id,user_id,playcount
1,TRFUPBA128F934F7E1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
2,TRLQPQJ128F42AA94F,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
3,TRTUCUY128F92E1D24,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1


In [230]:
music_info[1:4]

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",NaN,2006,258613,0.409,...,2,-4.373,1,0.0336,0.000807,0.000000,0.2070,0.651,174.426,4
2,TROUVHL128F426C441,Come as You Are,Nirvana,https://p.scdn.co/mp3-preview/a1c11bb1cb231031...,0keNu0t0tqsWtExGM3nT1D,"rock, alternative, alternative_rock, 90s, grunge",RnB,1991,218920,0.508,...,4,-5.783,0,0.0400,0.000175,0.000459,0.0878,0.543,120.012,4
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,https://p.scdn.co/mp3-preview/399c401370438be4...,0ancVQ9wEcHVd0RrGICTE4,"rock, alternative, indie, alternative_rock, in...",NaN,2004,237026,0.279,...,9,-8.851,1,0.0371,0.000389,0.000655,0.1330,0.490,104.560,4


### Grouping the songs of each user

In [231]:
user_song_list = users_history.groupby('user_id', observed=True)[['track_id', 'playcount']].apply(lambda x: list(zip(x['track_id'], x['playcount']))).to_dict()

In [232]:
dict(list(user_song_list.items())[:5])

{'00000b722001882066dff9d2da8a775658053ea0': [('TRQEBOU128F425D087', 1)],
 '00001638d6189236866af9bbf309ae6c2347ffdc': [('TRBCDMC128F1452976', 1)],
 '0000175652312d12576d9e6b84f600caa24c4715': [('TRHVDTF128F428D64C', 1)],
 '00001cf0dce3fb22b0df0f3a1d9cd21e38385372': [('TRZPDFG128F92FDEF5', 2),
  ('TRKABMK128F426BF72', 2),
  ('TRGEIDA128F933B4B8', 1),
  ('TRTPXAP12903CA1A14', 2),
  ('TRIYKXW128F92FDF08', 2),
  ('TRVRIBZ128F426BF71', 1),
  ('TRPUGUW128F426BF6F', 1),
  ('TREZYWT128F93191B7', 2)],
 '0000267bde1b3a70ea75cf2b2d216cb828e3202b': [('TRPXIWX128F429831F', 1),
  ('TRJGDTG128F421CE22', 1),
  ('TRORDNM128F427560D', 1),
  ('TRLTISM128F424FEFB', 1),
  ('TRIVDZY12903CC7671', 1),
  ('TREXNUE128F92FFC12', 1),
  ('TRODQGV128F425FD7E', 1),
  ('TRSMXVB128F42858BC', 1),
  ('TRXVQAJ128F9316B65', 1)]}

###  Removing the users who have listened to less than 50 songs

In [233]:
user_song_list = {user: songs for user, songs in user_song_list.items() if len(songs) >= 50}

In [234]:
dict(list(user_song_list.items())[:2])

{'0003a64f7a69e5b87a80b09c3772227185c235c7': [('TRRCWXH128F42B917C', 1),
  ('TRMHZLT12903CEA646', 1),
  ('TROVIQH128F42B91A1', 1),
  ('TRDNHAW128F429DB9A', 1),
  ('TRBVSQT128F427A82F', 1),
  ('TRAFUNV128F92CFEB2', 2),
  ('TRRXFHO128EF3550BC', 1),
  ('TRKQLRS128F4230F04', 1),
  ('TRBVNWT128F93173BA', 1),
  ('TRYYDLF128F423BB17', 1),
  ('TRUWRWK128F92E16B6', 1),
  ('TRMPTXE128F9303E92', 1),
  ('TRNGKOQ128E0780DAC', 1),
  ('TRVKPXM128F4284E60', 1),
  ('TRHTYWL128F429EFD8', 1),
  ('TRBROBA128F9321BC2', 3),
  ('TRFNDHN128F426BE40', 1),
  ('TRAVRKY128F429EA68', 1),
  ('TRILTQH128F93264C4', 1),
  ('TRUTKSF128F147B0E2', 1),
  ('TRBXLFU128F427A494', 1),
  ('TRRUZLF128F42396D0', 1),
  ('TRBMCGS128F92D2C27', 5),
  ('TROBYUK128F9335CA7', 1),
  ('TRKRADU128F92E17F6', 1),
  ('TRJGWMZ128F149F0BD', 1),
  ('TRGVKBK128F429EA2D', 1),
  ('TRIFCFC128F4230F13', 1),
  ('TRJLVOB128F931EB19', 1),
  ('TRJDWKM128E07817F1', 1),
  ('TRNPEXS128F1460294', 5),
  ('TRCYSEC128F428B5E8', 1),
  ('TRTBNOP12903CDB267', 6),

In [235]:
len(user_song_list)

23795

### Removing the data related to users that have listened to less than 50 songs

In [236]:
users_history = users_history[users_history['user_id'].isin(user_song_list.keys())] 
users_history.shape

(1808122, 3)

In [237]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix,coo_matrix
from annoy import AnnoyIndex

### Selecting the columns that represent the numerical values for each song

In [240]:
feature_columns = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 
                   'instrumentalness', 'liveness', 'valence', 'tempo']
#numerical_features = music_info[feature_columns].values



### Normalizing the features

In [241]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
normalized_numerical_features = scaler.fit_transform(music_info[feature_columns])

### Building the annoy index

In [242]:
feature_length = normalized_numerical_features.shape[1]
annoy_index = AnnoyIndex(feature_length, 'angular')

for idx, vector in enumerate(normalized_numerical_features):
    annoy_index.add_item(idx, vector)

annoy_index.build(50)

True

### The 6 most close songs to the first song

In [243]:
annoy_index.get_nns_by_item(0, 7)[1:]

[24376, 19933, 42612, 42620, 17844, 1202]

### Splitting the data

In [244]:
train, test = train_test_split(users_history, test_size=0.2, random_state=42)

### Creating mappings so we can find a song_id by index and vice versa

In [245]:
# For collaborative filtering (using users_history)
train['user_id'] = train['user_id'].astype('category')
train['track_id'] = train['track_id'].astype('category')

# Create mappings for user_id and track_id
cf_user_id_mapping = dict(enumerate(train['user_id'].cat.categories))
cf_track_id_mapping = dict(enumerate(train['track_id'].cat.categories))
cf_user_id_reverse_mapping = {v: k for k, v in cf_user_id_mapping.items()}
cf_track_id_reverse_mapping = {v: k for k, v in cf_track_id_mapping.items()}

# For content-based filtering (using music_info)
music_info['track_id'] = music_info['track_id'].astype('category')

cb_track_id_mapping = dict(enumerate(music_info['track_id'].cat.categories))
cb_track_id_reverse_mapping = {v: k for k, v in cb_track_id_mapping.items()}



### Using SVD

In [246]:
# Create Sparse User-Item Interaction Matrix
user_item_sparse = coo_matrix((train['playcount'],
                               (train['user_id'].cat.codes,
                                train['track_id'].cat.codes)))

# Apply SVD on the Sparse Matrix
svd = TruncatedSVD(n_components=10, random_state=42)
user_factors = svd.fit_transform(user_item_sparse)
item_factors = svd.components_.T

In [247]:
user_item_sparse.shape

(23795, 26000)

In [248]:
def recommend_songs_hybrid(user_id, user_item_matrix, user_factors, item_factors, music_info, annoy_index, n_recommendations=5):
    # Check if the user_id exists in the mapping
    user_code = cf_user_id_reverse_mapping.get(user_id)
    if user_code is None:
        print(f"User ID {user_id} not found in the user-item matrix.")
        return []
    
    # Collaborative Filtering Recommendations
    cf_predictions = np.dot(user_factors[user_code, :], item_factors.T)
    cf_indices = np.argsort(cf_predictions)[::-1]
    cf_recommended_tracks = [cf_track_id_mapping[i] for i in cf_indices[:n_recommendations]]
    
    #print(f"Collaborative Filtering Recommendations for user {user_id}: {cf_recommended_tracks}")
    
    # Content-Based Filtering Recommendations
    cb_recommended_tracks = []
    for track in cf_recommended_tracks:
        track_code = cb_track_id_reverse_mapping.get(track)
        if track_code is not None:
            similar_tracks = annoy_index.get_nns_by_item(track_code, 4)[1:]  # Get 4 similar tracks
            for i in similar_tracks:
                try:
                    cb_recommended_tracks.append(cb_track_id_mapping[i])
                except KeyError:
                    print(f"KeyError: Index {i} not found in cb_track_id_mapping")
    
    #print(f"Content-Based Filtering Recommendations for user {user_id}: {cb_recommended_tracks}")
    
    hybrid_recommended_tracks = list(set(cf_recommended_tracks + cb_recommended_tracks))
    
    #print(f"Hybrid Recommendations for user {user_id}: {hybrid_recommended_tracks}")
    
    return hybrid_recommended_tracks

In [249]:
def evaluate_model_hybrid_user(user, user_test_data, user_train_data, user_item_matrix, user_factors, item_factors, music_info, annoy_index, n_recommendations=5):
    precision = 0.0
    recall = 0.0
    if user in user_train_data:
            true_tracks = user_test_data[user]
            recommended_tracks = recommend_songs_hybrid(user, user_item_matrix, user_factors, item_factors, music_info, annoy_index, n_recommendations)
            
            # Calculate precision and recall
            true_positives = len(set(recommended_tracks) & set(true_tracks))
            precision = true_positives / len(recommended_tracks) if recommended_tracks else 0
            recall = true_positives / len(true_tracks) if true_tracks else 0
    return precision, recall        
            
        

In [250]:
def evaluate_model_hybrid(user_test_data, user_train_data, user_item_matrix, user_factors, item_factors, music_info, annoy_index, n_recommendations=5):
    precisions = []
    recalls = []

    for user, true_tracks in user_test_data.items():
        if user in user_train_data:
            recommended_tracks = recommend_songs_hybrid(user, user_item_matrix, user_factors, item_factors, music_info, annoy_index, n_recommendations)
            
            # Calculate precision and recall
            true_positives = len(set(recommended_tracks) & set(true_tracks))
            precision = true_positives / len(recommended_tracks) if recommended_tracks else 0
            recall = true_positives / len(true_tracks) if true_tracks else 0
            
            precisions.append(precision)
            recalls.append(recall)
    
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    
    return avg_precision, avg_recall

In [251]:
user_train_data = train.groupby('user_id', observed=True)['track_id'].apply(list).to_dict()
user_test_data = test.groupby('user_id', observed=True)['track_id'].apply(list).to_dict()

In [252]:
#user_train_data["0000f88f8d76a238c251450913b0d070e4a77d19"]

In [253]:
#precision, recall = evaluate_model_hybrid_user("0000f88f8d76a238c251450913b0d070e4a77d19", user_test_data, user_train_data, user_item_sparse, user_factors, item_factors, music_info, annoy_index, n_recommendations=5)
#print(f"Average Precision: {precision}")
#print(f"Average Recall: {recall}")

In [254]:
#recommend_songs_hybrid("0000f88f8d76a238c251450913b0d070e4a77d19", user_item_sparse, user_factors, item_factors, music_info, annoy_index, n_recommendations=5)

### Evaluating the model

In [255]:
precision, recall = evaluate_model_hybrid(user_test_data, user_train_data, user_item_sparse, user_factors, item_factors, music_info, annoy_index, 11)
print(f"Average Precision: {precision}")
print(f"Average Recall: {recall}")

Average Precision: 0.009004503317848413
Average Recall: 0.027114453080296717
